In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import seaborn as sns
import re

In [21]:
#Read in the original csv file
NY511 = pd.read_csv('../data/511_NY_Events__Beginning_2010.csv', low_memory=False)

#Data source: https://catalog.data.gov/nl/dataset/511-ny-events-beginning-2010

In [41]:
NY511.info() #2627317 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627317 entries, 0 to 2627316
Data columns (total 13 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Event Type                  object 
 1   Organization Name           object 
 2   Facility Name               object 
 3   Direction                   object 
 4   City                        object 
 5   County                      object 
 6   State                       object 
 7   Create Time                 object 
 8   Close Time                  object 
 9   Event Description           object 
 10  Responding Organization Id  object 
 11  Latitude                    float64
 12  Longitude                   float64
dtypes: float64(2), object(11)
memory usage: 260.6+ MB


In [22]:
#List of counties with confirmed Brood 5 (2016) and Brood 8 (2018) sightings
NYcounties = ['Cayuga', 'Onondaga', 'Livingston','Suffolk']

In [23]:
#New DataFrame with relevant counties only
NY511_trim = NY511[(NY511['County'].isin(NYcounties))]

## Getting Relevant Traffic Incident Types


In [81]:
NY511_trim['Event Type'].unique()

array(['specialevents', 'construction', 'guard rail repairs',
       'expansion joint repairs', 'roadwork', 'bridge work',
       'overhead sign repair', 'roving repairs', 'accident', 'incident',
       'pothole repairs', 'drainage improvements', 'line striping',
       'construction, culvert repairs', 'milling, repaving',
       'disabled vehicle', 'pothole repair', 'road sweeping',
       'deck work, expansion joint repairs', 'disabled tractor trailer',
       'road rehabilitation', 'overturned vehicle', 'steel repairs',
       'traffic signal repairs', 'deck work',
       'police department activity', 'emergency construction',
       'downed tree', 'bridge rehabilitation, line striping',
       'guard rail repairs, installation of sign structure, pothole repair, roadwork',
       'vehicle fire', 'culvert repairs', 'bridge rehabilitation',
       'delays, stop and go traffic',
       'installation of conduits, vms repairs', 'milling',
       'delays, earlier incident', 'repaving', 'a

In [82]:
#Relevant'Event Type' values to keep
#assumptions made: These are unplanned events typically caused by human(s) action.  
road_events=['accident', 'incident', 'delays, stop and go traffic', 
'delays, earlier incident', 'debris spill', 'earlier incident', 'special event', 
'accident, delays', 'delays', 'accident road closed', 'downed wires, downed pole', 'downed pole',
'accident, disabled vehicle', 'delays, stop and go traffic, traffic congestion',
'debris spill, earlier incident', 'accident, overturned tractor trailer', 'accident, emergency maintenance', 
'delays, heavy traffic', 'overturned tractor trailer', 'accident, jack-knifed tractor trailer', 
'jack-knifed tractor trailer', 'accident, overturned truck', 'overturned truck', 'stop and go traffic, traffic congestion',
'rubbernecking delays', 'accident, debris spill']


In [83]:
#trim dataframe to match the list of indivents/event types
NY511_trim = NY511_trim[(NY511_trim['Event Type'].isin(road_events))]

In [84]:
NY511_trim.info()  #55011 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19497 entries, 55011 to 1272921
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  19497 non-null  object        
 1   Organization Name           19497 non-null  object        
 2   Facility Name               19497 non-null  object        
 3   Direction                   18624 non-null  object        
 4   City                        18399 non-null  object        
 5   County                      19497 non-null  object        
 6   State                       19497 non-null  object        
 7   Create Time                 19497 non-null  datetime64[ns]
 8   Close Time                  19282 non-null  object        
 9   Event Description           19497 non-null  object        
 10  Responding Organization Id  19497 non-null  object        
 11  Latitude                    19497 non-null  floa

## Getting Relevant Years and Months

In [131]:
#convert times to datetimes
NY511_trim['Create Time'] = pd.to_datetime(NY511_trim['Create Time'])

In [132]:
#Periodical cicadas are confirmed in NY for Brood 5 (2016) and Brood 8 (2018) - trimmed extra years out.  Keep 2015, 2016, 2017, 2018, 2019.
NY511_trim = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2015,1,1)]
NY511_trim = NY511_trim[NY511_trim['Create Time'] < dt.datetime(2020,1,1)]

In [133]:
NY511_trim.info() #19497 entries.  Jan 2015 - Jan 2020

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19497 entries, 55011 to 1272921
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  19497 non-null  object        
 1   Organization Name           19497 non-null  object        
 2   Facility Name               19497 non-null  object        
 3   Direction                   18624 non-null  object        
 4   City                        18399 non-null  object        
 5   County                      19497 non-null  object        
 6   State                       19497 non-null  object        
 7   Create Time                 19497 non-null  datetime64[ns]
 8   Close Time                  19282 non-null  object        
 9   Event Description           19497 non-null  object        
 10  Responding Organization Id  19497 non-null  object        
 11  Latitude                    19497 non-null  floa

In [13]:
#ATTEMPT 1:
#Reducing records to May and June of the years 2015, 2016, 2017, 2018, 2019.  Cicada months.

#df[(df['date'] > '2002-1-1 01:00:00') & (df['date'] <= '2002-1-1 04:00:00')]
#mask = (df['Joined date'] > '2019-06-1') & (df['Joined date'] <= '2020-02-05')

NY511_trimtest = (NY511_trim['Create Time'].dt.month == 5) & (NY511_trim['Create Time'].dt.month == 6)
NY511_trimtest.tail(60) 

# returns 'False' instead of row values...

In [134]:
#Breaking apart the datetime to date and time (don't need to include the time)
#df['new_date'] = [d.date() for d in df['my_timestamp']]
NY511_trim['Date'] = [d.date() for d in NY511_trim['Create Time']]

In [135]:
#Attempt 2:
##Reducing records to May and June of the years 2015, 2016, 2017, 2018, 2019.  Cicada months.

NY511_2015 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2015,5,1)]
NY511_2015 = NY511_2015[NY511_trim['Create Time'] < dt.datetime(2015,6,30)]

<ipython-input-135-ae078b57c5f1>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NY511_2015 = NY511_2015[NY511_trim['Create Time'] < dt.datetime(2015,6,30)]


In [136]:
NY511_2015.info() #474 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474 entries, 1207693 to 1252169
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  474 non-null    object        
 1   Organization Name           474 non-null    object        
 2   Facility Name               474 non-null    object        
 3   Direction                   452 non-null    object        
 4   City                        425 non-null    object        
 5   County                      474 non-null    object        
 6   State                       474 non-null    object        
 7   Create Time                 474 non-null    datetime64[ns]
 8   Close Time                  473 non-null    object        
 9   Event Description           474 non-null    object        
 10  Responding Organization Id  474 non-null    object        
 11  Latitude                    474 non-null    floa

In [137]:
NY511_2016 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2016,5,1)]
NY511_2016 = NY511_2016[NY511_2016['Create Time'] < dt.datetime(2016,6,30)]

In [122]:
NY511_2016.info() #1176 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176 entries, 829109 to 920702
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  1176 non-null   object        
 1   Organization Name           1176 non-null   object        
 2   Facility Name               1176 non-null   object        
 3   Direction                   1115 non-null   object        
 4   City                        1044 non-null   object        
 5   County                      1176 non-null   object        
 6   State                       1176 non-null   object        
 7   Create Time                 1176 non-null   datetime64[ns]
 8   Close Time                  1174 non-null   object        
 9   Event Description           1176 non-null   object        
 10  Responding Organization Id  1176 non-null   object        
 11  Latitude                    1176 non-null   float

In [138]:
NY511_2017 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2017,5,1)]
NY511_2017 = NY511_2017[NY511_trim['Create Time'] < dt.datetime(2017,6,30)]

<ipython-input-138-da6f987a6b93>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NY511_2017 = NY511_2017[NY511_trim['Create Time'] < dt.datetime(2017,6,30)]


In [124]:
NY511_2017.info() #753 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 573156 to 619589
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  753 non-null    object        
 1   Organization Name           753 non-null    object        
 2   Facility Name               753 non-null    object        
 3   Direction                   719 non-null    object        
 4   City                        740 non-null    object        
 5   County                      753 non-null    object        
 6   State                       753 non-null    object        
 7   Create Time                 753 non-null    datetime64[ns]
 8   Close Time                  752 non-null    object        
 9   Event Description           753 non-null    object        
 10  Responding Organization Id  753 non-null    object        
 11  Latitude                    753 non-null    float6

In [139]:
NY511_2018 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2018,5,1)]
NY511_2018 = NY511_2018[NY511_trim['Create Time'] < dt.datetime(2018,6,30)]

<ipython-input-139-0d3dcf07edf6>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NY511_2018 = NY511_2018[NY511_trim['Create Time'] < dt.datetime(2018,6,30)]


In [103]:
NY511_2018.info()  #737 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 321253 to 367478
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  737 non-null    object        
 1   Organization Name           737 non-null    object        
 2   Facility Name               737 non-null    object        
 3   Direction                   723 non-null    object        
 4   City                        713 non-null    object        
 5   County                      737 non-null    object        
 6   State                       737 non-null    object        
 7   Create Time                 737 non-null    datetime64[ns]
 8   Close Time                  734 non-null    object        
 9   Event Description           737 non-null    object        
 10  Responding Organization Id  737 non-null    object        
 11  Latitude                    737 non-null    float6

In [140]:
NY511_2019 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2019,5,1)]
NY511_2019 = NY511_2019[NY511_trim['Create Time'] < dt.datetime(2019,6,30)]

<ipython-input-140-2511484711d7>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NY511_2019 = NY511_2019[NY511_trim['Create Time'] < dt.datetime(2019,6,30)]


In [105]:
NY511_2019.info() #0 entries.  There might be a gap in the data. See below

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  0 non-null      object        
 1   Organization Name           0 non-null      object        
 2   Facility Name               0 non-null      object        
 3   Direction                   0 non-null      object        
 4   City                        0 non-null      object        
 5   County                      0 non-null      object        
 6   State                       0 non-null      object        
 7   Create Time                 0 non-null      datetime64[ns]
 8   Close Time                  0 non-null      object        
 9   Event Description           0 non-null      object        
 10  Responding Organization Id  0 non-null      object        
 11  Latitude                    0 non-null      float64       
 12  Longit

In [115]:
#Why no rows for 2019?
NY511_trim.sort_values('Create Time', ascending=False)
NY511_trim.head(1200)
#No rows between 2019-07-01 00:11:15 and 2019-04-03 07:56:17.  Data in data collection? Bug from my trimming?

,Event Type,Organization Name,Facility Name,Direction,City,County,State,Create Time,Close Time,Event Description,Responding Organization Id,Latitude,Longitude
55011,accident,New York State Department of Transportation Re...,I-690,Westbound,before Geddes,Onondaga,NY,2019-09-30 16:20:20,09/30/2019 05:17:24 PM,Accident on I-690 westbound before Exit 7 - RO...,New York State Department of Transportation Re...,43.076400,-76.214302
55058,accident,New York State Department of Transportation (R...,I-495,Eastbound,between Huntington,Suffolk,NY,2019-09-30 15:32:19,09/30/2019 04:10:18 PM,Crash on I-495 eastbound between Exit 49N - NY...,New York State Department of Transportation (R...,40.780799,-73.417701
55061,accident,New York State Department of Transportation Re...,I-81,Southbound,before Syracuse,Onondaga,NY,2019-09-30 15:29:18,09/30/2019 03:58:14 PM,Accident on I-81 southbound before Exit 17 - S...,New York State Department of Transportation Re...,43.020000,-76.140296
55161,incident,MTA Long Island Rail Road,Montauk Branch,Westbound,from Suffolk County,Suffolk,NY,2019-09-30 13:27:17,09/30/2019 02:09:18 PM,"Due to Track work, westbound Montauk Branch fr...",MTA Long Island Rail Road,41.045433,-71.951011
55268,incident,New York State Department of Transportation (R...,I-495,Eastbound,between Huntington,Suffolk,NY,2019-09-30 10:46:20,09/30/2019 02:18:22 PM,Crash road closed on I-495 eastbound between E...,New York State Department of Transportation (R...,40.787700,-73.378402
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128501,incident,MTA Long Island Rail Road,Montauk Branch,Westbound,out of Suffolk County,Suffolk,NY,2019-07-01 03:19:17,07/01/2019 03:52:18 AM,"Due to Equipment problems, westbound Montauk B...",MTA Long Island Rail Road,41.045433,-71.951011
128512,incident,New York State Department of Transportation (R...,NY 25,Westbound,area of Huntington,Suffolk,NY,2019-07-01 02:01:15,07/01/2019 04:29:14 AM,Downed wires on NY 25 westbound area of Commac...,New York State Department of Transportation (R...,40.842899,-73.293197
128537,incident,MTA Long Island Rail Road,Montauk Branch,Westbound,from Suffolk County,Suffolk,NY,2019-07-01 00:11:15,07/01/2019 02:01:15 AM,"Due to Equipment problems, westbound Montauk B...",MTA Long Island Rail Road,40.820976,-72.704284
128724,incident,MTA Long Island Rail Road,Port Jefferson Branch,Westbound,from Suffolk County,Suffolk,NY,2019-04-03 07:56:17,NaN,"Due to Operational activity, westbound Port Je...",MTA Long Island Rail Road,40.852352,-73.409812


---------------------------------

--------------------------

## Export the data

In [144]:
## Bringing the yearly dataframes into a single dataframe

years = [NY511_2015, NY511_2016, NY511_2017, NY511_2018, NY511_2019]
NY511_final = pd.concat(years)

In [145]:
#Create a column specifically for the date

NY511_final['Date'] = [d.date() for d in NY511_final['Create Time']]
NY511_final['Date'] = pd.to_datetime(NY511_final['Date'])

In [146]:
#Save new dataframe as a csv
NY511_final.to_csv('NY511_final.csv', index = False, header=True)